In [4]:
import pandas as pd

# 替换为你本地 parquet 文件路径
file_path = "fhvhv_tripdata_2024-01.parquet"

# 读取 Parquet 文件（需先安装 pyarrow：pip install pyarrow）
df = pd.read_parquet(file_path)

# 显示数据集column
print(df.columns)


Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')


In [9]:
import pandas as pd
import numpy as np
import glob

# 获取所有 parquet 文件路径
parquet_files = glob.glob("./fhvhv_tripdata_2024-*.parquet")


# 读取并合并
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file, columns=["pickup_datetime", "PULocationID"])
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
    df["pickup_hour"] = df["pickup_datetime"].dt.floor("H")
    df_list.append(df)

df_all = pd.concat(df_list, ignore_index=True)

# 按区域 + 小时 聚合订单数
region_hourly = df_all.groupby(["PULocationID", "pickup_hour"]).size().reset_index(name="trip_count")

# 添加时间特征
region_hourly["hour"] = region_hourly["pickup_hour"].dt.hour
region_hourly["weekday"] = region_hourly["pickup_hour"].dt.dayofweek
region_hourly["is_weekend"] = (region_hourly["weekday"] >= 5).astype(int)
region_hourly["is_peak_hour"] = region_hourly["hour"].isin([7, 8, 9, 17, 18, 19]).astype(int)

# 节假日标记（如需更多节日可替换）
holiday_list = ["2024-01-01", "2024-05-27", "2024-07-04"]
region_hourly["is_holiday"] = region_hourly["pickup_hour"].dt.date.astype(str).isin(holiday_list).astype(int)

# 🔁 按区域分别构造滞后和滚动特征
def add_lag_features(df, group_key="PULocationID"):
    df = df.sort_values(by=[group_key, "pickup_hour"])
    df["lag_1"] = df.groupby(group_key)["trip_count"].shift(1)
    df["lag_24"] = df.groupby(group_key)["trip_count"].shift(24)
    df["rolling_mean_6"] = df.groupby(group_key)["trip_count"].shift(1).rolling(6).mean().reset_index(0, drop=True)
    df["rolling_mean_24"] = df.groupby(group_key)["trip_count"].shift(1).rolling(24).mean().reset_index(0, drop=True)
    return df

region_hourly = add_lag_features(region_hourly)
region_hourly = region_hourly.dropna().reset_index(drop=True)

# 💾 导出
region_hourly.to_csv("region_hourly_order_features_full.csv", index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8904\3201619385.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["pickup_hour"] = df["pickup_datetime"].dt.floor("H")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8904\3201619385.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["pickup_hour"] = df["pickup_datetime"].dt.floor("H")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8904\3201619385.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["pickup_hour"] = df["pickup_datetime"].dt.floor("H")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8904\3201619385.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["pickup_hour"] = df["pickup_datetime"].dt.floor("H")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8904\3201619385.py:14: FutureWarning: 'H' is deprecated and